In [4]:
from dataset import create_dataset, create_sampler, create_loader
from models.model_retrieval import XVLM

In [1]:
config = {
    'train_file': ['data/finetune/cosmos_caption_test.json'], 
    'val_file': ['data/finetune/cosmos_caption_test.json'], 
    'test_file': ['data/finetune/cosmos_caption_test.json'],
    'image_res': 384,
    'image_root': '/root/thesis/dataset',
    'prompt':'',
    'max_tokens': 4,
    'patch_size': 32,
    'use_clip_vit': False,
    'use_swin':True,
    'vision_config': 'configs/config_swinB_384.json',
    'use_roberta': False,
    'text_config': 'configs/config_bert.json',  # ['configs/config_bert.json', 'configs/config_roberta.json']
'text_encoder': 'data/bert-base-uncased' ,
'batch_size_train': 20,
'batch_size_test': 12,
'batch_size_test_text': 64,
'max_tokens': 40,
'embed_dim': 256,
'temp': 0.07,
'k_test': 256
}

In [5]:
model = XVLM(config=config)
# model.load_pretrained('/root/thesis/X-VLM/checkpoint/checkpoint_best_refcoco.pth', config, is_eval=True)

In [6]:
from models.tokenization_bert import BertTokenizer
from tqdm import tqdm
tokenizer = BertTokenizer.from_pretrained(config['text_encoder'])

In [7]:
train_dataset, val_dataset, test_dataset = create_dataset('caption_cosmos', config);
datasets = [train_dataset, val_dataset, test_dataset]

/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:834: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(
/root/anaconda3/envs/xvlm/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [8]:
train_loader, val_loader, test_loader = create_loader(datasets, [None, None, None], batch_size=[5] * 3,
                                                          num_workers=[4, 4, 4], is_trains=[True, False, False],
                                                          collate_fns=[None, None, None])

In [15]:
device = 'cuda:0'
model.to(device);

In [16]:
itc=[]
itm=[]
for img,cap,label in tqdm(train_loader):
    img = img.to(device)
    text_inputs = tokenizer(cap, padding='longest', return_tensors="pt",max_length=512, truncation=True).to(device)
    pred = model(img,text_inputs.input_ids, text_inputs.attention_mask)
    itc.append(pred[0].item())
    itm.append(pred[1].item())

  0%|          | 0/200 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
100%|██████████| 200/200 [00:39<00:00,  5.05it/s]


In [21]:
import numpy as np
np.max(itm)

0.789966344833374